# Rock, Paper, Scissors

## Imports

In [18]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense
from keras.callbacks import TensorBoard
from keras.utils import to_categorical
import datetime

## Verbose
If this flag is set to true, cells will give more detailed output

In [19]:
verbose = True

## Read csv

In [20]:
data = pd.read_csv('/home/carol/Documents/Master/3_Semester/Anwendung der KI/Project/data.csv')
if verbose:
    print(data.head())

  C1 H1 C2 H2 C3 H3 C4 H4 C5 H5 C6 H6
0  P  R  P  S  R  R  P  S  S  S  S  S
1  R  R  R  R  S  R  R  R  P  R  R  S
2  S  S  S  S  S  R  S  S  P  R  S  S
3  P  S  R  S  P  R  P  S  S  S  S  S
4  R  R  S  R  R  R  R  S  R  S  P  R


## Transforming the data into numerical values

In [21]:
# P = 0; R = 1; S = 2
label_encoder = LabelEncoder()
for column in data.columns:
    data[column] = label_encoder.fit_transform(data[column])

if verbose: 
    print(data.head())  


   C1  H1  C2  H2  C3  H3  C4  H4  C5  H5  C6  H6
0   0   1   0   2   1   1   0   2   2   2   2   2
1   1   1   1   1   2   1   1   1   0   1   1   2
2   2   2   2   2   2   1   2   2   0   1   2   2
3   0   2   1   2   0   1   0   2   2   2   2   2
4   1   1   2   1   1   1   1   2   1   2   0   1


## Making input and Output 
- The input data is an array with the picks of the player and the computer in the last n rounds. The amount of rounds if the defined in the variable sequence_length
- The output data, what the computer is trying to guess, is the next human pick.

In [22]:
sequence_length = 3
data_number = 1
X = []
y = []

for i in range(len(data) - sequence_length):
    X.append(data[[f'H{data_number}', f'C{data_number}']].iloc[i:i+sequence_length].values)
    y.append(data['H1'].iloc[i + sequence_length])

X = np.array(X)
y = np.array(y)

if verbose:
    print(X[0])
    print(y[0])
    print(X[1])
    print(y[1])
    print(X[2])
    print(y[2])


[[1 0]
 [1 1]
 [2 2]]
2
[[1 1]
 [2 2]
 [2 0]]
1
[[2 2]
 [2 0]
 [1 1]]
2


## Spliting the data into train and test set

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Changing the output data to hot encoding

In [24]:
output_dim = len(label_encoder.classes_)
y_train = to_categorical(y_train, num_classes=output_dim)
y_test = to_categorical(y_test, num_classes=output_dim)

if verbose:
    print(y_train[0])

[0. 0. 1.]


## Tune the Model

In [25]:
hiddenUnits = 10
stackLSTM = True # The model will include Dropout and hiddenUnits layers if True
dropout = 0.2
input_dim = X_train.shape[2] # Human and Computer
timestep_length = X_train.shape[1] # 5 last rounds
if verbose: 
    print(f'input_dim: {input_dim}')
    print(f'timestep_length: {timestep_length}')

input_dim: 2
timestep_length: 3


In [26]:
model = Sequential()
model.add(LSTM(hiddenUnits, return_sequences=stackLSTM, input_shape=(timestep_length, input_dim)))
if stackLSTM:
    model.add(Dropout(dropout))
    model.add(LSTM(hiddenUnits))
model.add(Dropout(dropout))
model.add(Dense(output_dim, activation='softmax'))

/home/carol/Documents/Master/3_Semester/Anwendung der KI/Project/.env/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


## Train

In [27]:
now = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tbCallBack = TensorBoard(log_dir='./logs/' + now, histogram_freq=1, write_graph=True, write_images=False)

showProgress = 0
if verbose: 
    showProgress = 2

model.compile(loss='categorical_crossentropy')
model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=32,
    verbose=showProgress,
    validation_split=0.2,
    callbacks=[tbCallBack]
)


Epoch 1/100
1/1 - 4s - 4s/step - loss: 1.1706 - val_loss: 1.1270
Epoch 2/100
1/1 - 0s - 159ms/step - loss: 1.1703 - val_loss: 1.1249
Epoch 3/100
1/1 - 0s - 169ms/step - loss: 1.1514 - val_loss: 1.1232
Epoch 4/100
1/1 - 0s - 153ms/step - loss: 1.1432 - val_loss: 1.1218
Epoch 5/100
1/1 - 0s - 151ms/step - loss: 1.1371 - val_loss: 1.1205
Epoch 6/100
1/1 - 0s - 129ms/step - loss: 1.1324 - val_loss: 1.1195
Epoch 7/100
1/1 - 0s - 123ms/step - loss: 1.1210 - val_loss: 1.1185
Epoch 8/100
1/1 - 0s - 115ms/step - loss: 1.1188 - val_loss: 1.1176
Epoch 9/100
1/1 - 0s - 115ms/step - loss: 1.1017 - val_loss: 1.1170
Epoch 10/100
1/1 - 0s - 118ms/step - loss: 1.1051 - val_loss: 1.1161
Epoch 11/100
1/1 - 0s - 106ms/step - loss: 1.0895 - val_loss: 1.1155
Epoch 12/100
1/1 - 0s - 110ms/step - loss: 1.0923 - val_loss: 1.1149
Epoch 13/100
1/1 - 0s - 111ms/step - loss: 1.0909 - val_loss: 1.1143
Epoch 14/100
1/1 - 0s - 114ms/step - loss: 1.0875 - val_loss: 1.1138
Epoch 15/100
1/1 - 0s - 111ms/step - loss: 1.0

## Evaluate the Model

In [28]:
loss = model.evaluate(X_test, y_test)
print(loss)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 1.0587
1.0586860179901123


## Predict the Next Move

In [29]:
new_moves = np.array([[[label_encoder.transform(['R'])[0], label_encoder.transform(['P'])[0]],
                       [label_encoder.transform(['P'])[0], label_encoder.transform(['R'])[0]],
                       [label_encoder.transform(['S'])[0], label_encoder.transform(['S'])[0]]]])  # Replace with actual new moves
predicted_move = model.predict(new_moves)
predicted_move = label_encoder.inverse_transform([np.argmax(predicted_move)])
print(f'Predicted next move: {predicted_move[0]}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step
Predicted next move: S


## Determine the counter move for the computer

### Function Implementation

In [30]:
def counter_move(player_move):
    if player_move == 'R':
        return 'P'  # Paper beats Rock
    elif player_move == 'P':
        return 'S'  # Scissors beats Paper
    elif player_move == 'S':
        return 'R'  # Rock beats Scissors

### Function Call

In [31]:
computer_move = counter_move(predicted_move[0])
print(f'Computer should play: {computer_move}')

Computer should play: R
